In [1]:
import xml.dom.minidom
import xml.etree.ElementTree as ET
from xml.dom.minidom import Node
import pandas as pd
import numpy as np

In [159]:
Historical_DOU_Fall_Counts = pd.read_csv("CY2020 DOU Inpatient Fall Summary.csv")
Q1 = Historical_DOU_Fall_Counts[Historical_DOU_Fall_Counts.Month.isin([1,2,3])]
Q2 = Historical_DOU_Fall_Counts[Historical_DOU_Fall_Counts.Month.isin([4,5,6])]
Q3 = Historical_DOU_Fall_Counts[Historical_DOU_Fall_Counts.Month.isin([7,8,9])]
Q4 = Historical_DOU_Fall_Counts[Historical_DOU_Fall_Counts.Month.isin([10,11,12])]

In [160]:
Historical_DOU_Fall_Details = pd.read_csv("CY2020 DOU Inpatient Fall Details.csv")
Q1_Details = Historical_DOU_Fall_Details[Historical_DOU_Fall_Details.Month.isin([1,2,3])]
Q2_Details = Historical_DOU_Fall_Details[Historical_DOU_Fall_Details.Month.isin([4,5,6])]
Q3_Details = Historical_DOU_Fall_Details[Historical_DOU_Fall_Details.Month.isin([7,8,9])]
Q4_Details = Historical_DOU_Fall_Details[Historical_DOU_Fall_Details.Month.isin([10,11,12])]

In [2]:
# This is the overall turnover summary data
Fall_data = pd.read_csv("2021_Q2 DOU Inpatient Fall Summary.csv")
# These are the individual turnover details
Individual_Falls = pd.read_csv("2021_Q2 DOU Inpatient Fall Details.csv")

In [3]:
# Fill NaN with 0 and change data type from float to integer
Fall_data.loc[:, ('FallsCount')] = Fall_data.loc[:, ('FallsCount')].fillna(0.0).astype(int)

In [4]:
# Change the patient age from float to integer but first make the null age a 0
Individual_Falls.Years = Individual_Falls.Years.fillna(0.0).astype(int)
# Change the patient age from float to integer but first make the null age a 0
Individual_Falls.FallRiskAssmntScore = Individual_Falls.FallRiskAssmntScore.fillna(0.0).astype(int)

In [5]:
# The variables needed for this function are the: Hospital abbreviation, Hospital NDNQI template, Hospital NDNQI
# code, Calendar year being submitted, and Calendar quarter being submitted
def NDNQI_Inpatient_Falls_XML(Selected_Hosp,Hosp_XML_Template,Hosp_NDNQI_Code,Calendar_Year,Calendar_Quarter):
    
    # This section is to update the total Fall Count Values
    
    # The first step takes only the data for this selected hospital
    Hosp_data = Fall_data[Fall_data.Hospital == Selected_Hosp]
    # A List of this Hospital's Unit Names
    Hosp_Units = list(set(Hosp_data.UnitName))

     #This is the current XML template for this hospital and their NDNQI units for this metric
    tree = xml.dom.minidom.parse(Hosp_XML_Template)
    # This is each unit node of the xml file
    unit_nodes = tree.getElementsByTagName("UnitYrQtr")

    # Make sure the correct NDNQI code, year, and quarter are listed
    H = tree.getElementsByTagName("HospitalCode")[0].childNodes[0].nodeValue = Hosp_NDNQI_Code
    Y = tree.getElementsByTagName("Year")[0].childNodes[0].nodeValue = Calendar_Year
    Q = tree.getElementsByTagName("Quarter")[0].childNodes[0].nodeValue = Calendar_Quarter


    # This loops through each separate UnitName from the Template XML file and replaces it WITH the correct NDNQI
    # name for the units. 
    for idx, unit in enumerate(unit_nodes):
        unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue = Hosp_Units[idx]

        # Now grab each of the 3 months for that unit from the Hospital data
        new_data = Hosp_data[Hosp_data.UnitName == Hosp_Units[idx]]
        # Sort the data by month
        new_data = new_data.sort_values('Month')

        # Take each xml Month and replace it with the months from the new_data for that unit
        month_nodes = unit.getElementsByTagName("UnitMonth")
        for idx2, month in enumerate(month_nodes):
            month.getElementsByTagName('Month')[0].childNodes[0].nodeValue = new_data.Month.iloc[idx2]

            # Now replace each xml metric value with data that has the same name from our data source
            for column in new_data.columns[3:]:

                new_value = new_data[column].iloc[idx2]
                month.getElementsByTagName(column)[0].childNodes[0].nodeValue = new_value



    # This is a temporary xml file that I will now use to add the individual separation data.
    tree.writexml(open('Temp_{}.xml'.format(Selected_Hosp), 'w'))

In [6]:
# The variables needed for this function are the: Hospital abbreviation, Hospital Temp XML file,
# Calendar year being submitted, and Calendar quarter being submitted
def NDNQI_InPt_Indiv_Falls_XML(Selected_Hosp,Temp_XML,Calendar_Year,Calendar_Quarter):
    # This section is to insert the individual falls. 

    # This Hospital's last Quarter's individual falls
    Hosp_individual = Individual_Falls[Individual_Falls.Hospital == Selected_Hosp]

    individual_falls = ['Years', 'Months', 'Days', 'xGenderFID', 'xFallTypeFID', 'xFallWasPhysiologicalYNFID',
            'xInjuryLevelFID', 'xPriorRiskAssmntFID', 'xFallRiskAssmntPatient', 'FallRiskAssmntScore', 
            'xRecencyHoursFID', 'xAtRiskFID', 'FallIdentifier', 'UnitFallFIDPrevious', 'xYesNoFIDFallAssisted',
            'xFallEmployeeTypeFID', 'FallObserved', 'FallWasSupervisedByPTorOT', 'xYesNoFIDPhysicalRestraints', 
            'xYesNoFIDFallProtocol']

    # This is the temporary xml file that has already had the "FallsCount" section filled
    tree = xml.dom.minidom.parse(Temp_XML)
    unit_nodes = tree.getElementsByTagName("UnitYrQtr")

    # This loops through each listed unit in our xml file
    for idx, unit in enumerate(unit_nodes):
        each_unit = unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue

        # Then we loop through each listed month
        month_nodes = unit.getElementsByTagName("UnitMonth")
        for idx2, month in enumerate(month_nodes):
            each_month = month.getElementsByTagName('Month')[0].childNodes[0].nodeValue

            # I first check to find if any falls were listed
            fall_count = month.getElementsByTagName('FallsCount')[0].childNodes[0].nodeValue
            intentional_fall_count = month.getElementsByTagName('IntentionalFallsCount')[0].childNodes[0].nodeValue

            all_falls = [int(fall_count), int(intentional_fall_count)]

            total_falls = sum(all_falls)

            # If a fall was listed for that month and that unit
            if total_falls > 0:

                # Then I grab this unit's data from Hosp_individual falls
                new_data = Hosp_individual[Hosp_individual.UnitName == each_unit]
                # Lastly, I select only that unit's separations that occured in this specific month
                each_fall = new_data[new_data.Month == int(each_month)]

                # Now I can take each listed fall
                for i in range(len(each_fall)):
                    # and create a "UnitFall" node for each individual fall
                    fall = tree.createElement("UnitFall")
                    for data in individual_falls:
                        # I create an element for each individual's metrics
                        data_name = tree.createElement(data)

                        if data in ['xFallRiskAssmntPatient', 'xRecencyHoursFID', 'xAtRiskFID', 'UnitFallFIDPrevious', 
                                    'FallWasSupervisedByPTorOT', 'xYesNoFIDPhysicalRestraints', 'xYesNoFIDFallProtocol'] and pd.isna(each_fall[data].iloc[i]):
                            data_name.appendChild(tree.createTextNode(''))
                        elif data == 'Years' and each_fall[data].iloc[i] == 0:
                            data_name.setAttribute('xsi:nil', 'true')
                        elif data == 'FallRiskAssmntScore' and each_fall[data].iloc[i] == 0:
                            data_name.setAttribute('xsi:nil', 'true')
                        elif pd.isna(each_fall[data].iloc[i]):
                            data_name.setAttribute('xsi:nil', 'true')
                        else:
                            # and I add each one to that individual's "UnitFall" node
                            data_name.appendChild(tree.createTextNode(str(each_fall[data].iloc[i])))

                        fall.appendChild(data_name)

                    # Finally, I can add these individual falls to the "UnitFalls" section.
                    month.getElementsByTagName("UnitFalls")[0].appendChild(fall)


    # Now, we need to write and save this XML file as its own NDNQI Quarter and Year and Hospital to upload
    # into NDNQI.
    tree.writexml(open('Q{}_{} {} Inpatient Falls.xml'.format(Calendar_Quarter, Calendar_Year,Selected_Hosp), 'w'))

## EUOSH

In [7]:
NDNQI_Inpatient_Falls_XML('EUOSH','2020 EUOSH Inpatient Falls Template.xml','SBI556',2021,2)

In [8]:
NDNQI_InPt_Indiv_Falls_XML('EUOSH','Temp_EUOSH.xml',2021,2)

In [9]:
# This Hospital's Turnover Summary Data
EUOSH_data = Fall_data[Fall_data.Hospital == 'EUOSH']
# This Hospital's last Quarter's individual falls
EUOSH_individual = Individual_Falls[Individual_Falls.Hospital == 'EUOSH']

In [12]:
EUOSH_data

,Hospital,UnitName,Month,FallsCount,IntentionalFallsCount,xFallRiskAssmntFID
12,EUOSH,OSH ICU,4,0,0,Morse
13,EUOSH,OSH ICU,5,0,0,Morse
14,EUOSH,OSH ICU,6,1,0,Morse
27,EUOSH,OSH4 / OSH3,4,1,0,Morse
28,EUOSH,OSH4 / OSH3,5,0,0,Morse
29,EUOSH,OSH4 / OSH3,6,1,0,Morse
153,EUOSH,OSH2,4,0,0,Morse
154,EUOSH,OSH2,5,0,0,Morse
155,EUOSH,OSH2,6,1,0,Morse


In [11]:
EUOSH_individual

,Hospital,UnitName,Month,Years,Months,Days,xGenderFID,xFallTypeFID,xFallWasPhysiologicalYNFID,xYesNoFIDFallAssisted,...,xRecencyHoursFID,xAtRiskFID,UnitFallFIDPrevious,FallWasSupervisedByPTorOT,xYesNoFIDFallProtocol,FallIdentifier,xPriorRiskAssmntFID,FallObserved,xFallEmployeeTypeFID,xYesNoFIDPhysicalRestraints
30,EUOSH,OSH2,6,50,NaN,NaN,Male,Other,NaN,No,...,> 0 to 12 hours,NaN,NaN,NaN,No,202162804,Yes,NaN,NaN,NaN
57,EUOSH,OSH4 / OSH3,6,80,NaN,NaN,Male,Other,NaN,No,...,> 0 to 12 hours,NaN,NaN,NaN,Yes,202161987,Yes,Yes,NaN,NaN
93,EUOSH,OSH ICU,6,58,NaN,NaN,Female,Other,NaN,No,...,> 0 to 12 hours,NaN,NaN,NaN,Yes,202161109,Yes,NaN,NaN,NaN
290,EUOSH,OSH4 / OSH3,4,68,NaN,NaN,Female,Other,NaN,Yes,...,NaN,NaN,NaN,Yes,Yes,202156855,No Documentation,Yes,Physical or Occupational Therapist,NaN


## EJCH

In [13]:
NDNQI_Inpatient_Falls_XML('EJCH','2020 EJCH Inpatient Falls Template.xml','YTD069',2021,2)

In [14]:
NDNQI_InPt_Indiv_Falls_XML('EJCH','Temp_EJCH.xml',2021,2)

In [15]:
# This Hospital's Turnover Summary Data
EJCH_data = Fall_data[Fall_data.Hospital == 'EJCH']
# This Hospital's last Quarter's individual falls
EJCH_individual = Individual_Falls[Individual_Falls.Hospital == 'EJCH']

In [17]:
EJCH_data

,Hospital,UnitName,Month,FallsCount,IntentionalFallsCount,xFallRiskAssmntFID
39,EJCH,JC ICU,4,0,0,Morse
40,EJCH,JC ICU,5,0,0,Morse
41,EJCH,JC ICU,6,1,0,Morse
48,EJCH,JC4,4,5,0,Morse
49,EJCH,JC4,5,5,0,Morse
50,EJCH,JC4,6,1,0,Morse
51,EJCH,JC Neonatal ICU,4,0,0,Morse
52,EJCH,JC Neonatal ICU,5,0,0,Morse
53,EJCH,JC Neonatal ICU,6,0,0,Morse
105,EJCH,JC MB,4,0,0,Morse


In [16]:
EJCH_individual

,Hospital,UnitName,Month,Years,Months,Days,xGenderFID,xFallTypeFID,xFallWasPhysiologicalYNFID,xYesNoFIDFallAssisted,...,xRecencyHoursFID,xAtRiskFID,UnitFallFIDPrevious,FallWasSupervisedByPTorOT,xYesNoFIDFallProtocol,FallIdentifier,xPriorRiskAssmntFID,FallObserved,xFallEmployeeTypeFID,xYesNoFIDPhysicalRestraints
28,EJCH,JC3,6,76,NaN,NaN,Female,Other,NaN,No,...,> 0 to 12 hours,NaN,NaN,NaN,No,202162852,Yes,NaN,NaN,NaN
33,EJCH,JC4,6,89,NaN,NaN,Male,Other,NaN,No,...,> 0 to 12 hours,NaN,NaN,NaN,Yes,202162629,Yes,NaN,NaN,NaN
81,EJCH,JC ICU,6,54,NaN,NaN,Female,Other,NaN,No,...,> 0 to 12 hours,NaN,NaN,NaN,No,202161439,Yes,Yes,NaN,NaN
87,EJCH,JC Oncology,6,81,NaN,NaN,Male,Other,NaN,No,...,> 0 to 12 hours,NaN,NaN,NaN,No,202161353,Yes,NaN,NaN,NaN
132,EJCH,JC3,5,87,NaN,NaN,Female,Other,NaN,No,...,> 0 to 12 hours,NaN,NaN,NaN,No,202160260,Yes,NaN,NaN,NaN
137,EJCH,JC3,5,82,NaN,NaN,Male,Other,NaN,No,...,> 0 to 12 hours,NaN,NaN,NaN,Yes,202160222,Yes,NaN,NaN,NaN
174,EJCH,JC4,5,44,NaN,NaN,Male,Other,NaN,No,...,NaN,NaN,NaN,NaN,Yes,202159272,No,NaN,NaN,NaN
177,EJCH,JC Oncology,5,71,NaN,NaN,Female,Other,NaN,Yes,...,> 12 to 24 hours,NaN,NaN,NaN,Yes,202159261,Yes,Yes,Nursing Staff,NaN
184,EJCH,JC3,5,32,NaN,NaN,Female,Other,NaN,No,...,NaN,NaN,NaN,NaN,No,202159139,No Documentation,NaN,NaN,NaN
190,EJCH,JC Oncology,5,66,NaN,NaN,Female,Other,NaN,No,...,> 0 to 12 hours,NaN,NaN,NaN,Yes,202159040,Yes,NaN,NaN,NaN


## ESJH

In [18]:
NDNQI_Inpatient_Falls_XML('ESJH','2020 ESJH Inpatient Falls Template.xml','XAR317',2021,2)

In [19]:
NDNQI_InPt_Indiv_Falls_XML('ESJH','Temp_ESJH.xml',2021,2)

In [20]:
# This Hospital's Turnover Summary Data
ESJH_data = Fall_data[Fall_data.Hospital == 'ESJH']
# This Hospital's last Quarter's individual falls
ESJH_individual = Individual_Falls[Individual_Falls.Hospital == 'ESJH']

In [21]:
ESJH_data

,Hospital,UnitName,Month,FallsCount,IntentionalFallsCount,xFallRiskAssmntFID
18,ESJH,2 West ICU,4,1,0,Morse
19,ESJH,2 West ICU,5,0,0,Morse
20,ESJH,2 West ICU,6,0,0,Morse
21,ESJH,2 East ICU,4,1,0,Morse
22,ESJH,2 East ICU,5,1,0,Morse
23,ESJH,2 East ICU,6,1,0,Morse
24,ESJH,5 East Internal Medicine,4,1,0,Morse
25,ESJH,5 East Internal Medicine,5,2,0,Morse
26,ESJH,5 East Internal Medicine,6,2,0,Morse
30,ESJH,ESJH 7 East Oncology,4,3,0,Morse


In [22]:
ESJH_individual.head()

,Hospital,UnitName,Month,Months,Days,xFallTypeFID,xFallWasPhysiologicalYNFID,xYesNoFIDFallAssisted,xInjuryLevelFID,xFallRiskAssmntPatient,...,UnitFallFIDPrevious,FallWasSupervisedByPTorOT,xYesNoFIDFallProtocol,FallIdentifier,xPriorRiskAssmntFID,FallObserved,xFallEmployeeTypeFID,Years,xYesNoFIDPhysicalRestraints,xGenderFID
9,ESJH,4 South Telemetry (Orthopedic),3,NaN,NaN,Other,NaN,No,Minor,Morse,...,NaN,NaN,No,202156175,Yes,NaN,NaN,31,NaN,Female
10,ESJH,2 East ICU,3,NaN,NaN,Other,NaN,No,None,Morse,...,NaN,NaN,No,202156139,Yes,NaN,NaN,77,NaN,Male
12,ESJH,6 East Telemetry (Internal medicine),3,NaN,NaN,Other,NaN,Yes,None,Morse,...,NaN,NaN,Yes,202156115,Yes,Yes,Nursing Staff,75,NaN,Male
20,ESJH,2 South CV Step-down,3,NaN,NaN,Other,NaN,No,Minor,Morse,...,NaN,NaN,Yes,202155975,Yes,NaN,NaN,73,NaN,Female
21,ESJH,3 West Cardiology,3,NaN,NaN,Other,NaN,No,None,Morse,...,NaN,NaN,No,202155961,Yes,Yes,NaN,65,NaN,Male


## EUHM

In [22]:
NDNQI_Inpatient_Falls_XML('EUHM','2020 EUHM Inpatient Falls Template.xml','PVF155',2021,2)

In [23]:
NDNQI_InPt_Indiv_Falls_XML('EUHM','Temp_EUHM.xml',2021,2)

In [24]:
# This Hospital's Turnover Summary Data
EUHM_data = Fall_data[Fall_data.Hospital == 'EUHM']
# This Hospital's last Quarter's individual falls
EUHM_individual = Individual_Falls[Individual_Falls.Hospital == 'EUHM']

In [25]:
EUHM_data

,Hospital,UnitName,Month,FallsCount,IntentionalFallsCount,xFallRiskAssmntFID
0,EUHM,U61 Med (COVID-19),4,6,0,Morse
1,EUHM,U61 Med (COVID-19),5,3,0,Morse
2,EUHM,U61 Med (COVID-19),6,3,0,Morse
9,EUHM,72 Stroke/Neuro,4,0,0,Morse
10,EUHM,72 Stroke/Neuro,5,2,0,Morse
11,EUHM,72 Stroke/Neuro,6,3,0,Morse
42,EUHM,U52 Renal,4,3,0,Morse
43,EUHM,U52 Renal,5,3,0,Morse
44,EUHM,U52 Renal,6,3,0,Morse
60,EUHM,71 ICU,4,0,0,Morse


In [26]:
EUHM_individual.head()

,Hospital,UnitName,Month,Years,Months,Days,xGenderFID,xFallTypeFID,xFallWasPhysiologicalYNFID,xYesNoFIDFallAssisted,...,xRecencyHoursFID,xAtRiskFID,UnitFallFIDPrevious,FallWasSupervisedByPTorOT,xYesNoFIDFallProtocol,FallIdentifier,xPriorRiskAssmntFID,FallObserved,xFallEmployeeTypeFID,xYesNoFIDPhysicalRestraints
0,EUHM,71 ICU,6,72,NaN,NaN,Male,Other,NaN,Yes,...,NaN,NaN,NaN,NaN,Yes,202163416,No Documentation,Yes,Nursing Staff,NaN
4,EUHM,U61 Med (COVID-19),6,52,NaN,NaN,Female,Other,NaN,No,...,> 0 to 12 hours,NaN,NaN,NaN,Yes,202163330,Yes,NaN,NaN,NaN
6,EUHM,21 ICU,6,59,NaN,NaN,Male,Other,NaN,No,...,> 0 to 12 hours,NaN,NaN,NaN,Yes,202163297,Yes,NaN,NaN,NaN
8,EUHM,72 Stroke/Neuro,6,62,NaN,NaN,Male,Other,NaN,No,...,> 0 to 12 hours,NaN,NaN,NaN,No,202163232,Yes,NaN,NaN,NaN
17,EUHM,22,6,72,NaN,NaN,Male,Other,NaN,No,...,NaN,NaN,NaN,NaN,Yes,202163099,No,NaN,NaN,NaN


## EUH

In [27]:
NDNQI_Inpatient_Falls_XML('EUH','2020 EUH Inpatient Falls Template.xml','JXE432',2021,2)

In [28]:
NDNQI_InPt_Indiv_Falls_XML('EUH','Temp_EUH.xml',2021,2)

In [29]:
# This Hospital's Turnover Summary Data
EUH_data = Fall_data[Fall_data.Hospital == 'EUH']
# This Hospital's last Quarter's individual falls
EUH_individual = Individual_Falls[Individual_Falls.Hospital == 'EUH']

In [33]:
set(list(EUH_data.UnitName))

{'10E Medical',
 '2D/2G ICU',
 '3G Neuro',
 '3G/4G CCU',
 '4G Cardiac',
 '5D Med',
 '5E CVICU',
 '5G ARICU',
 '5G Medical',
 '5T N ICU',
 '5T-S ICU',
 '6B Rollins',
 '6E CV',
 '6G Medical',
 '6T Trans',
 '7E Gen Med',
 '7G (Medical)',
 '7T Surg',
 '8E Step Down',
 '8T N Onc',
 '8T S Hematology',
 '9 E Neuro',
 '9T BMT',
 'HG Inpatient',
 'WW Inpatient Unit'}

In [38]:
EUH_data[EUH_data.UnitName == '9T BMT']

,Hospital,UnitName,Month,FallsCount,IntentionalFallsCount,xFallRiskAssmntFID
57,EUH,9T BMT,4,3,0,Morse
58,EUH,9T BMT,5,1,0,Morse
59,EUH,9T BMT,6,10,0,Morse


In [39]:
EUH_individual.head()

,Hospital,UnitName,Month,Years,Months,Days,xGenderFID,xFallTypeFID,xFallWasPhysiologicalYNFID,xYesNoFIDFallAssisted,...,xRecencyHoursFID,xAtRiskFID,UnitFallFIDPrevious,FallWasSupervisedByPTorOT,xYesNoFIDFallProtocol,FallIdentifier,xPriorRiskAssmntFID,FallObserved,xFallEmployeeTypeFID,xYesNoFIDPhysicalRestraints
1,EUH,2D/2G ICU,6,45,NaN,NaN,Female,Other,NaN,No,...,> 0 to 12 hours,NaN,NaN,NaN,Yes,202163394,Yes,NaN,NaN,NaN
2,EUH,5T-S ICU,6,0,NaN,NaN,Female,Other,NaN,Yes,...,NaN,NaN,NaN,NaN,No,202163372,No Documentation,Yes,Other Hospital Staff,NaN
3,EUH,WW Inpatient Unit,6,81,NaN,NaN,Female,Other,NaN,No,...,> 0 to 12 hours,NaN,NaN,NaN,Yes,202163362,Yes,NaN,NaN,NaN
5,EUH,WW Inpatient Unit,6,81,NaN,NaN,Female,Other,NaN,No,...,> 0 to 12 hours,NaN,NaN,NaN,No,202163327,Yes,Yes,NaN,NaN
7,EUH,7T Surg,6,61,NaN,NaN,Female,Other,NaN,No,...,NaN,NaN,NaN,NaN,Yes,202163296,No,NaN,NaN,NaN


## EDH

In [7]:
NDNQI_Inpatient_Falls_XML('EDH','2021 EDH Inpatient Falls Template.xml','NMW346',2021,2)

In [8]:
NDNQI_InPt_Indiv_Falls_XML('EDH','Temp_EDH.xml',2021,2)

In [9]:
# This Hospital's Turnover Summary Data
EDH_data = Fall_data[Fall_data.Hospital == 'EDH']
# This Hospital's last Quarter's individual falls
EDH_individual = Individual_Falls[Individual_Falls.Hospital == 'EDH']

In [10]:
EDH_data

,Hospital,UnitName,Month,FallsCount,IntentionalFallsCount,xFallRiskAssmntFID
3,EDH,EDH 25 ICU,4,0,0,Hendrich II
4,EDH,EDH 25 ICU,5,0,0,Hendrich II
5,EDH,EDH 25 ICU,6,0,0,Hendrich II
6,EDH,EDH 44,4,3,0,Hendrich II
7,EDH,EDH 44,5,3,0,Hendrich II
8,EDH,EDH 44,6,4,0,Hendrich II
9,EDH,EDH 22,4,0,0,Hendrich II
10,EDH,EDH 22,5,0,0,Hendrich II
11,EDH,EDH 22,6,1,0,Hendrich II
18,EDH,EDH 32/33,4,4,0,Hendrich II


In [11]:
EDH_individual.head()

,Hospital,UnitName,Month,Years,Months,Days,xFallTypeFID,xFallWasPhysiologicalYNFID,xInjuryLevelFID,xPriorRiskAssmntFID,...,xYesNoFIDFallAssisted,FallObserved,FallWasSupervisedByPTorOT,FallIdentifier,xYesNoFIDPhysicalRestraints,xGenderFID,xRecencyHoursFID,xFallEmployeeTypeFID,FallRiskAssmntScore,xYesNoFIDFallProtocol
5,EDH,EDH 24,5,79,NaN,NaN,Other,NaN,None,Yes,...,No,NaN,NaN,43655,NaN,Male,> 0 to 12 hours,NaN,7,NaN
6,EDH,EDH 24,5,77,NaN,NaN,Other,NaN,Minor,Yes,...,No,NaN,NaN,43667,Yes,Male,> 0 to 12 hours,NaN,10,NaN
7,EDH,EDH 24,5,57,NaN,NaN,Other,NaN,None,Yes,...,Yes,NaN,NaN,43699,NaN,Male,> 0 to 12 hours,Nursing Staff,6,NaN
8,EDH,EDH 24,5,58,NaN,NaN,Other,NaN,None,Yes,...,No,NaN,NaN,43741,NaN,Female,> 0 to 12 hours,NaN,3,NaN
9,EDH,EDH 24,5,60,NaN,NaN,Other,NaN,None,Yes,...,No,NaN,NaN,43889,NaN,Female,> 0 to 12 hours,NaN,4,NaN


## EHH

In [12]:
NDNQI_Inpatient_Falls_XML('EHH','2021 EHH Inpatient Falls Template.xml','WXX116',2021,2)

In [13]:
NDNQI_InPt_Indiv_Falls_XML('EHH','Temp_EHH.xml',2021,2)

In [14]:
# This Hospital's Turnover Summary Data
EHH_data = Fall_data[Fall_data.Hospital == 'EHH']
# This Hospital's last Quarter's individual falls
EHH_individual = Individual_Falls[Individual_Falls.Hospital == 'EHH']

In [15]:
EHH_data

,Hospital,UnitName,Month,FallsCount,IntentionalFallsCount,xFallRiskAssmntFID
0,EHH,EHH 5,4,0,0,Hendrich II
1,EHH,EHH 5,5,2,0,Hendrich II
2,EHH,EHH 5,6,3,0,Hendrich II
12,EHH,EHH 4 ICU,4,0,0,Hendrich II
13,EHH,EHH 4 ICU,5,0,0,Hendrich II
14,EHH,EHH 4 ICU,6,0,0,Hendrich II
15,EHH,EHH 3,4,4,0,Hendrich II
16,EHH,EHH 3,5,1,0,Hendrich II
17,EHH,EHH 3,6,0,0,Hendrich II
33,EHH,EHH 4,4,4,0,Hendrich II


In [16]:
EHH_individual.head()

,Hospital,UnitName,Month,Years,Months,Days,xFallTypeFID,xFallWasPhysiologicalYNFID,xInjuryLevelFID,xPriorRiskAssmntFID,...,xYesNoFIDFallAssisted,FallObserved,FallWasSupervisedByPTorOT,FallIdentifier,xYesNoFIDPhysicalRestraints,xGenderFID,xRecencyHoursFID,xFallEmployeeTypeFID,FallRiskAssmntScore,xYesNoFIDFallProtocol
0,EHH,EHH 3,4,49,NaN,NaN,Other,NaN,None,Yes,...,No,NaN,NaN,42020,NaN,Female,> 12 to 24 hours,NaN,8,NaN
1,EHH,EHH 3,4,60,NaN,NaN,Other,NaN,Minor,No,...,No,NaN,NaN,42058,NaN,Male,NaN,NaN,0,NaN
2,EHH,EHH 3,4,31,NaN,NaN,Other,NaN,None,Yes,...,No,NaN,NaN,42317,NaN,Female,> 0 to 12 hours,NaN,1,NaN
3,EHH,EHH 3,4,50,NaN,NaN,Other,NaN,None,Yes,...,No,NaN,NaN,43388,NaN,Male,> 0 to 12 hours,NaN,2,NaN
4,EHH,EHH 3,5,68,NaN,NaN,Other,NaN,None,Yes,...,No,NaN,NaN,43456,NaN,Female,> 0 to 12 hours,NaN,8,NaN


## ELTAC

In [17]:
NDNQI_Inpatient_Falls_XML('ELTAC','2021 ELTAC Inpatient Falls Template.xml','XLS464',2021,2)

In [18]:
NDNQI_InPt_Indiv_Falls_XML('ELTAC','Temp_ELTAC.xml',2021,2)

In [19]:
# This Hospital's Turnover Summary Data
ELTAC_data = Fall_data[Fall_data.Hospital == 'ELTAC']
# This Hospital's last Quarter's individual falls
ELTAC_individual = Individual_Falls[Individual_Falls.Hospital == 'ELTAC']

In [20]:
ELTAC_data

,Hospital,UnitName,Month,FallsCount,IntentionalFallsCount,xFallRiskAssmntFID
21,ELTAC,ELTAC 5 ICU,4,0,0,Hendrich II
22,ELTAC,ELTAC 5 ICU,5,0,0,Hendrich II
23,ELTAC,ELTAC 5 ICU,6,0,0,Hendrich II
51,ELTAC,ELTAC 4/5,4,2,0,Hendrich II
52,ELTAC,ELTAC 4/5,5,2,0,Hendrich II
53,ELTAC,ELTAC 4/5,6,2,0,Hendrich II


In [21]:
ELTAC_individual

,Hospital,UnitName,Month,Years,Months,Days,xFallTypeFID,xFallWasPhysiologicalYNFID,xInjuryLevelFID,xPriorRiskAssmntFID,...,xYesNoFIDFallAssisted,FallObserved,FallWasSupervisedByPTorOT,FallIdentifier,xYesNoFIDPhysicalRestraints,xGenderFID,xRecencyHoursFID,xFallEmployeeTypeFID,FallRiskAssmntScore,xYesNoFIDFallProtocol
10,ELTAC,ELTAC 4/5,4,29,NaN,NaN,Other,NaN,Minor,Yes,...,No,NaN,NaN,42260,Yes,Male,> 0 to 12 hours,NaN,13,NaN
11,ELTAC,ELTAC 4/5,4,67,NaN,NaN,Other,NaN,None,Yes,...,No,NaN,NaN,42344,NaN,Male,> 0 to 12 hours,NaN,12,NaN
12,ELTAC,ELTAC 4/5,5,41,NaN,NaN,Other,NaN,None,No,...,Yes,Yes,NaN,43435,NaN,Male,NaN,Physical or Occupational Therapist,0,NaN
13,ELTAC,ELTAC 4/5,5,77,NaN,NaN,Other,NaN,None,Yes,...,No,NaN,NaN,43739,Yes,Male,> 0 to 12 hours,NaN,10,NaN
14,ELTAC,ELTAC 4/5,6,73,NaN,NaN,Other,NaN,None,Yes,...,No,NaN,NaN,44105,NaN,Female,> 24 to 48 hours,NaN,12,NaN
15,ELTAC,ELTAC 4/5,6,76,NaN,NaN,Other,NaN,None,Yes,...,No,NaN,NaN,44102,Yes,Female,> 0 to 12 hours,NaN,10,NaN
